# Demonstration Notebook 

This is a notebook demonstrating how we can use `vigipy` to process the CAERS dataset. This should generalise to any dataset, but Pyspark could be useful to handle the largest databases (e.g. FAERS or Yellow Card)  
CAERS is the FDA database of adverse reactions associated with vitamins and food supplements.  
The steps in the MGPS algorithm are:
- Take in a database of drug-adverse event reports. These are reported in as pairs, so one patient may have many rows associated with them. This should normally come in as a table with columns for `patient_id`, `product_name`, `ae_name` and any stratification variables (like age, sex)
- There may be supplementary tables that contain further detailed patient information, drug outcomes, indications etc 
- This table should be processed to count all the drug-event pairs that appear in the database.
- This count method can be extended to also account for drug-drug interactions (or higher order terms)
- This table (known as a contingency table) should show how many times each drug-event pair appears in the database, how many time that drug appears (maybe with other events), how many times that event appears (maybe with other drugs) and the total number of reports in the database
- That contingency table is processed with the MGPS algorithm to produce an EBGM score for each drug-event pair (or drug-drug-event combiniation). This involves estimating hyperparameters for the distribution of events over the database by maximising a likelihood function.
- These hyperparameters are then used to calculate EBGM scores for each drug-event combination and any of them that are above a certain threshold are *suspicious* enough that they should be look at in more detail

## Running this notebook for yourself  
If you want to run this notebook, you will need to make sure that you have installed `vigipy` and also have access to the CAERS dataset.    
The dataset is currently stored on Azure in `/dbfs/mnt/data/caers/` if you would like to get it for yourself. It is open-source and fully anonymised.    
Please see the `README.md` for instructions on installing `vigipy`. I **strongly** recommend installing it in a virtual environment so you can play with it  
***IMPORTANT***: Make sure you are using the `local_qol_changes` branch because that is what I produced these results using.

In [1]:
from vigipy import * 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import time as time

In [2]:
caers_dataset = "C:/Users/damlteam/Documents/vigipy_devops/vigipy/example_notebooks/test_datasets/caers_dataset.csv" # put your own path to the dataset

# just reading in the CAERS dataset
df = pd.read_csv(caers_dataset, header = 0)
df.rename(columns={'var1': 'name'}, inplace=True)
df.rename(columns={'var2': 'AE'}, inplace=True)
df['count'] = 1

# drop duplicates from dataset 
df = df.drop_duplicates(subset=['id', 'name', 'AE'], keep='first')

In [3]:
df

,id,name,AE,strat1,count
0,147289,PREVAGEN,BRAIN NEOPLASM,Female,1
1,147289,PREVAGEN,CEREBROVASCULAR ACCIDENT,Female,1
2,147289,PREVAGEN,RENAL DISORDER,Female,1
3,147289,PREVAGEN,GOUT,Female,1
4,147289,PREVAGEN,HYPERTENSION,Female,1
...,...,...,...,...,...
20151,160591,"CENTRUM SILVER WOMEN'S 50 PLUS (MULTIMINERALS,...",CHOKING,Female,1
20152,160592,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL...",CHOKING,Female,1
20153,160592,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL...",PALPITATIONS,Female,1
20154,160592,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL...",DYSPHAGIA,Female,1


## Converting into a contingency table

There are really two ways to convert this table into a contingency table. One of these is the method originally used by `vigipy` and the other is the method originally used by `openEBGM`.   
The difference is in how many times you count a drug or adverse event appearing in the database. 
- If you have a patient who is taking 5 drugs and experiences 1 AE, then that will appear in the SRS as 5 drug-event pairs for each of the drugs and the AE
- When you are counting the number of times that that particular AE appeared in the database, do you count it once (because it occurred in one patient) or 5 times (because it occurred in 5 reports)?
- `OpenEBGM` counts it once, `vigipy` counts it 5 times.
- I have implemented this in `vigipy` as an optional flag called `count_unique_ids`. If this is set to `True`, then each AE/drug will only be counted *once per patient*. 
- I have optimised both the `openEBGM` style way and the `vigipy` style way of calculating this quantity within the code
- Generally, I think `count_unique_ids` should be set to `True`, because this makes more intuitive sense to me
- However, this is clearly debateable and probably should be discussed with PSM when we have that meeting
  

In [4]:
time1 = time.time()
vigipy_data_1 = convert(df, count_unique_ids=False, opt=False) # converting in the vigipy way, not using my optimisation
time2 = time.time()
print("TIME TO CONVERT WITH VIGIPY METHOD = ", time2-time1) # about 50 times slower than the optimised way

time1 = time.time()
vigipy_data_1_opt = convert(df, count_unique_ids=False, opt=True) # converting in the vigipy way, not using my optimisation
time2 = time.time()
print("TIME TO CONVERT WITH VIGIPY METHOD = ", time2-time1) # about 50 times quicker than the unoptimised way


TIME TO CONVERT WITH VIGIPY METHOD =  23.61515212059021
TIME TO CONVERT WITH VIGIPY METHOD =  0.51265549659729


In [5]:
time1 = time.time()
vigipy_data_2 = convert(df, count_unique_ids=True) # converting in the openEBGM way (always optimised)
time2 = time.time()
print("TIME TO CONVERT WITH OPENEBGM METHOD = ", time2-time1)

TIME TO CONVERT WITH OPENEBGM METHOD =  0.21882081031799316


In [6]:
# visualising the contingency table 
vigipy_data_1.contingency

# 2874 rows because 2874 different products 
# 1328 columns because 1328 different AEs reported
# most elements are zero because most product-AE events were never observed 


AE,ABASIA,ABDOMINAL DISCOMFORT,ABDOMINAL DISTENSION,ABDOMINAL PAIN,ABDOMINAL PAIN LOWER,ABDOMINAL PAIN UPPER,ABDOMINAL TENDERNESS,ABNORMAL BEHAVIOUR,ABNORMAL DREAMS,ABNORMAL FAECES,...,WHEEZING,WHITE BLOOD CELL COUNT DECREASED,WHITE BLOOD CELL COUNT INCREASED,WITHDRAWAL SYNDROME,WOUND HAEMORRHAGE,WOUND INFECTION STAPHYLOCOCCAL,WRONG DRUG ADMINISTERED,WRONG TECHNIQUE IN DRUG USAGE PROCESS,X-RAY ABNORMAL,YELLOW SKIN
name,,,,,,,,,,,,,,,,,,,,,
1-PHENYLALANINE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11 UNSPECIFIED VITAMINS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14-DAY ACAI BERRY CLEANSE,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2TX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4LIFE TRANSFER FACTOR TRI FACTOR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZINC CAPSULE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZINC DRINK,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZINC LOZENGE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# this contingency table is the same for both vigipy method and openebgm method 
((vigipy_data_1.contingency == vigipy_data_2.contingency).all(axis=0)).all()

np.True_

In [8]:
# Comparing what the two processed datasets look like 
data_comp1 = vigipy_data_1.data.sort_values(by='events')
data_comp2 = vigipy_data_2.data.sort_values(by='events')

In [9]:
data_comp1 # processed table using vigipy method of counts 

,events,product_aes,count_across_brands,ae_name,product_name
17184,1,18,1,X-RAY ABNORMAL,PURITAN'S PRIDE SAW PALMETTO 1000MG SOFTGELS
17153,1,13,49,WHITE BLOOD CELL COUNT INCREASED,MASS 2 HARDCORE
16506,1,11,24,VERTIGO,HERBAL CLEANSE DIETARY SUPPLEMENT
17155,1,14,49,WHITE BLOOD CELL COUNT INCREASED,NATUREMADE FISH OIL 1200MG
17156,1,21,49,WHITE BLOOD CELL COUNT INCREASED,PROBIOTIC
...,...,...,...,...,...
3528,27,75,1083,CHOKING,CENTRUM SILVER ULTRA WOMEN'S MULTIMINERALS MUL...
7540,28,169,383,FOREIGN BODY TRAUMA,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL..."
3563,42,104,1083,CHOKING,CENTRUM SILVER WOMEN'S 50+ MULTIMINERALS MULTI...
3537,53,146,1083,CHOKING,CENTRUM SILVER ULTRA WOMENS MULTIMINERALS MULT...


In [10]:
data_comp2 # processed table using openEBGM method of counts

,events,product_aes,count_across_brands,AE,name
17184,1,1,70,PALPITATIONS,ZINC LOZENGE
16410,1,2,37,SYNCOPE,VITAMIN WORLD NIACIN 500 MG COATED CAPLETS
16411,1,2,56,TREMOR,VITAMIN WORLD NIACIN 500 MG COATED CAPLETS
17155,1,9,5,FACE OEDEMA,ZINC
16412,1,2,2,VASCULAR INJURY,VITAMIN WORLD NIACIN 500 MG COATED CAPLETS
...,...,...,...,...,...
2358,27,30,842,CHOKING,CENTRUM SILVER ULTRA WOMEN'S MULTIMINERALS MUL...
2614,28,64,265,FOREIGN BODY TRAUMA,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL..."
2652,42,45,842,CHOKING,CENTRUM SILVER WOMEN'S 50+ MULTIMINERALS MULTI...
2430,53,57,842,CHOKING,CENTRUM SILVER ULTRA WOMENS MULTIMINERALS MULT...


It should be clear that in both the processed tables, the number of times each drug-event combination appears does not change at all.   
The change is in the `product_aes` which is the number of times that the specified product appears in the database and the `count_across_brands` which is the number of times that the specified AE appears in the database

## Optimising Hyperparameters and Building a Report
  
Once you have the processed data, you can use it to calculate the hyperparameters for the likelihood function and then use these to calculate a report. This is all handled by a very complicated function in `vigipy` called `gps` which has lots of steps. See the `src/GPS/GPS.py` file for the actual source code.   
I have not changed the core algorithm from the original GPS method, but I have refactored the code to make it readable, added a ridiculous amount of documentation (with the help of my trusty assistant who shall rename nameless) and made numerous optimisations throughout the code. 

### Optimising the likelihood function
  
The original `vigipy` was unusably slow for a relatively large dataset (like CAERS here). A lot of the work I've done with `vigipy` is in making it less slow and significantly more usable. This fundamentally came down to optimising the likelihood function. Here, I'm going to show what happens with the different optimised and unoptimised likelihood functions.  
  
There is a logical flag in `gps` called `opt_likelihood`. You can set this to `False` to use the original (unoptimised) likelihood functions. Please never do this unless you want the code to run for *hours*. 

In [11]:
## This step is just to process the data so you can see the impact of the optimised vs unoptimised likelihood functions
N = vigipy_data_2.N
data_cont = vigipy_data_2.contingency
data = vigipy_data_2.data

n11 = np.asarray(data["events"], dtype=np.float64)
n1j = np.asarray(data["product_aes"], dtype=np.float64)
ni1 = np.asarray(data["count_across_brands"], dtype=np.float64)
expected = calculate_expected(N, n1j, ni1, n11, 'mantel-haentzel', 1) # calculates the expected counts using the Mantel-Haentzel formula

n1__mat = data_cont.sum(axis=1)
n_1_mat = data_cont.sum(axis=0)
rep = len(n_1_mat)
n1__c = np.tile(n1__mat.values, reps=rep)
rep = len(n1__mat)
n_1_c = np.repeat(n_1_mat.values, repeats=rep)
E_c = np.asarray(n1__c, dtype=np.float64) * n_1_c / N
n11_c_temp = []
for col in data_cont:
    n11_c_temp.extend(list(data_cont[col]))
n11_c = np.asarray(n11_c_temp)

fake_priors = np.array([2,1,2,1,0.1]) # need to feed in fake values for the priors for this calculation

In [12]:
# This cell compares the performance of the optimised and the unoptimised likelihoods 
time1 = time.time()
likeli_unopt = non_truncated_likelihood(fake_priors, n11_c, E_c)
time2 = time.time()
print("TIME TAKEN TO CALCULATE LIKELIHOOD WITHOUT OPTIMISATION = ", time2-time1)

time1 = time.time()
likeli_opt = non_truncated_likelihood_optimised(fake_priors, n11_c, E_c)
time2 = time.time()
print("TIME TAKEN TO CALCULATE LIKELIHOOD WITHO OPTIMISATION = ", time2-time1)

print("Optimised likelihood = ", likeli_opt)
print("Unoptimised likelihood = ", likeli_unopt)

TIME TAKEN TO CALCULATE LIKELIHOOD WITHOUT OPTIMISATION =  45.081560373306274
TIME TAKEN TO CALCULATE LIKELIHOOD WITHO OPTIMISATION =  1.715895175933838
Optimised likelihood =  206700.16776762842
Unoptimised likelihood =  206698.43475301584


#### Effect of Optimisation
Some quite aggressive optimisation in the likelihood speeds up a single call to the likelihood function by a factor of ~50.  
This is mostly due to using `scipy` inbuilt functions rather than custom for-loop method of calculating the distribution functions.
In a single loop of an optimisation algorithm, the likelihood function is called 10 times (because 5 parameters and a gradient is calculation)    
This explains why this makes such a massive difference in run-times  
The final answers are (very nearly) the same. In fact, the optimised answer is more accurate!


#### Truncation
In practice, you should always use truncation in the dataset. This basically cuts out all values that are below a certain number. Remember that the contingency table was mostly zeroes. If you use the whole table, you have 1324x2874 terms in your likelihood function, which makes it expensive to calculate
Of those ~3 million terms, only 17000 are non-zero. So the safe (and best) thing to do is to just cut off all values that are smaller than 1. That is what truncation does. You *should* always use this, unless you have a reason to avoid it

In [13]:
# this shows the effect of truncation
truncate_thres = 1
truncate_number = truncate_thres - 1 # truncate_thres is the smallest value to keep in your dataset 
n11_truncated = n11[n11 >= truncate_thres]
expected_truncated = expected[n11 >= truncate_thres]

time1 = time.time()
likeli_trunc = truncated_likelihood(fake_priors, n11_truncated, expected_truncated, truncate_number)
time2 = time.time()
print("TIME TAKEN TO CALCULATE TRUNCATED LIKELIHOOD = ", time2-time1)

time1 = time.time()
likeli_trunc_opt = truncated_likelihood_optimised(fake_priors, n11_truncated, expected_truncated, truncate_number)
time2 = time.time()
print("TIME TAKEN TO CALCULATE TRUNCATED LIKELIHOOD WITH OPTIMISATION = ", time2-time1)

print("TRUNCATED LIKELIHOOD WITH OPTIMISATION = ", likeli_trunc_opt)
print("TRUNCATED LIKELIHOOD WITHOUT OPT = ", likeli_trunc)

TIME TAKEN TO CALCULATE TRUNCATED LIKELIHOOD =  0.7488925457000732
TIME TAKEN TO CALCULATE TRUNCATED LIKELIHOOD WITH OPTIMISATION =  0.01504826545715332
TRUNCATED LIKELIHOOD WITH OPTIMISATION =  4342.788567463042
TRUNCATED LIKELIHOOD WITHOUT OPT =  54820.55892349648


- Again, there are two versions (optimised and unoptimised) which correspond to the original `vigipy` and changes that I've mde
- You will notice that the unoptimised truncated likelihood gives a totally different answer to the optimised one. This is a proper error in vigipy. I can't work out where because it's hidden within loads of custom functions, but **do not** use this function. Only use the optimised version. The optimised version has been double checked against `openEBGM` and various test problems and is actually correct (and also quicker!)
- Also notice that the value of the likelihood function is completely different to the untruncated case (again, this is expected - the functions are literally different in the two cases)

### Calculating the hyperparameters
Once all this is done, we can finally discuss how the `gps` function works.  
See above for the description of the algorithm.  
It is all packaged up in the function, I will talk through the `src` code in a Monday meeting at some point...

In [14]:
help(gps) # help string for the function is very detailed

Help on function gps in module vigipy.GPS.GPS:

gps(container, relative_risk=1, min_events=1, decision_metric='rank', decision_thres=0.05, ranking_statistic='log2', truncate=False, truncate_thres=1, prior_init={'alpha1': 0.2041, 'beta1': 0.05816, 'alpha2': 1.415, 'beta2': 1.838, 'w': 0.0969}, prior_param=None, expected_method='mantel-haentzel', method_alpha=1, minimization_method='CG', minimization_bounds=((np.float32(1.1920929e-07), 20), (np.float32(1.1920929e-07), 10), (np.float32(1.1920929e-07), 20), (np.float32(1.1920929e-07), 10), (0, 1)), minimization_options=None, message=False, opt_likelihood=False, number_of_iterations=1000, tol_value=0.0001, sim_anneal=False, product_label='name', ae_label='AE')
    Perform disproportionality analysis using the Multi-Item Gamma Poisson Shrinker (GPS) algorithm.
    
    This function implements a gamma-poisson shrinker algorithm for analyzing adverse event 
    data and detecting disproportionality signals. It optimizes hyperparameters, calcu

In [15]:
# MHRA EBGM cut-off is 2.5
log2_ebgm = np.log2(2.5)

# bound for minimisation because values cannot go below zero
EPS = np.finfo(np.float64).eps

In [16]:
## not all of these parameters in GPS are needed!
time1 = time.time()
results = gps(
    vigipy_data_2, # container that was processed with convert
    relative_risk=1, # relative risk parameter
    min_events=3, # minimum number of events for something to be considered a signal 
    decision_metric='rank', # what should we rank signals by? 'rank' means by the ranking_statistic later 
    decision_thres=log2_ebgm, # minimum value of the ranking statistic for something to be considered a signal
    ranking_statistic='log2', # which ranking statistic to use, here log2 means the log2 of the EBGM score
    truncate=True, # whether to truncate or not
    truncate_thres=1,  # threshold for truncation
    prior_init={"alpha1": 0.2041, "beta1": 0.05816, "alpha2": 1.415, "beta2": 1.838, "w": 0.0969}, # initial guesses for priors
    prior_param=None, # feed in an array if you want to just use those priors and do no optimisation
    expected_method="mantel-haentzel", # method for calculating expected counts
    method_alpha=1, # parameter for other methods of calculating expected counts
    minimization_method="Nelder-Mead", # which minimisation algorithm from scipy to use!
    minimization_bounds=((EPS, 20), (EPS, 10), (EPS, 20), (EPS, 10), (EPS, 1)), # bounds on minimisation: will only be applied to certain algorithms
    minimization_options=None, # any supplementary options for the minimiser
    message=True, # whether to be verbose and print messages
    opt_likelihood=True, # whether to use the optimised versions of likelihood functions (really should always be true)
    number_of_iterations=1000, # number of iterations of the optimiser
    tol_value=1.0e-6, # tolerance for the optimiser
    sim_anneal=False, # whether to use simulated annealing optimisation: very experimental
    product_label='name', # the name of the column in the processed data that contains the product names
    ae_label='AE' # the name of the column in the processed data that contains the AE names
)
time2 = time.time()
print("TIME TAKEN TO PRODUCE DATA = ", time2-time1)

BEGINNING HYPERPARAMETER OPTIMISATION
OPTIMISED PRIORS REACHED:  [3.25597525 0.39999047 2.02375183 1.906132   0.06530732]
OPTIMISED FUNCTION VALUE =  4162.455710379452
Optimization terminated successfully.
CALCULATING EBGM SCORES
CALCULATING QUANTILES
GENERATING REPORT
TIME TAKEN TO PRODUCE DATA =  11.120262861251831


#### Observations
- Values of likelihood and priors that this gives are consistent with OpenEBGM for the same dataset!
- Values converge irrespective of which algorithm is used to minimise them, or the priors chosen (just affects the run-time)
- Various parameters are contained in the gps object (including all input parameters, optimised priors, likelihood at minimum)

In [17]:
# values are contained in the gps object 
results.__dict__

{'param': {'input_params': {'relative_risk': 1,
   'min_events': 3,
   'decision_metric': 'rank',
   'decision_thres': np.float64(1.3219280948873624),
   'ranking_statistic': 'log2',
   'truncate': True,
   'truncate_thres': 1,
   'prior_init': {'alpha1': 0.2041,
    'beta1': 0.05816,
    'alpha2': 1.415,
    'beta2': 1.838,
    'w': 0.0969},
   'prior_param': None,
   'expected_method': 'mantel-haentzel',
   'method_alpha': 1,
   'minimization_method': 'Nelder-Mead',
   'minimization_bounds': ((np.float64(2.220446049250313e-16), 20),
    (np.float64(2.220446049250313e-16), 10),
    (np.float64(2.220446049250313e-16), 20),
    (np.float64(2.220446049250313e-16), 10),
    (np.float64(2.220446049250313e-16), 1)),
   'minimization_options': None,
   'message': True,
   'opt_likelihood': True,
   'number_of_iterations': 1000,
   'tol_value': 1e-06,
   'sim_anneal': False,
   'product_label': 'name',
   'ae_label': 'AE'},
  'prior_init': {'alpha1': 0.2041,
   'beta1': 0.05816,
   'alpha2': 

In [18]:
results.all_signals 

,Product,Adverse Event,Count,Expected Count,log2,count/expected,product margin,event margin,fdr,FNR,Se,Sp,LowerBound,p_value
0,REUMOFAN PLUS,WEIGHT INCREASED,16.0,0.406436,4.539833,39.366569,44.0,31.0,0.079960,0.904489,0.879821,0.161140,15.686438,1.943874e-17
1,REUMOFAN PLUS,IMMOBILE,6.0,0.078665,4.191966,76.272727,44.0,6.0,0.080117,0.909704,0.861200,0.171486,10.161142,9.009766e-07
2,HYDROXYCUT REGULAR RAPID RELEASE CAPLETS,EMOTIONAL DISTRESS,19.0,0.896901,4.068400,21.184053,70.0,43.0,0.080021,0.900297,0.639618,0.422631,11.646773,2.541072e-17
3,"EMERGEN-C (ASCORBIC ACID, B-COMPLEX, ELECTROLY...",COUGH,6.0,0.144815,4.002402,41.432099,6.0,81.0,0.089050,0.925122,0.329246,0.568923,8.882635,2.815701e-06
4,HYDROXYCUT HARDCORE CAPSULES,MULTIPLE INJURIES,5.0,0.092372,3.966694,54.129032,31.0,10.0,0.086888,0.899664,0.572963,0.491910,8.271694,2.278501e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,"EMERGEN-C (ASCORBIC ACID, B-COMPLEX, ELECTROLY...",CHOKING,3.0,4.766985,-0.556960,0.629329,19.0,842.0,0.090537,0.925092,0.317295,0.578372,0.297589,7.915147e-01
435,FISH OIL,CHOKING,8.0,15.555423,-0.873756,0.514290,62.0,842.0,0.098179,0.923014,0.346318,0.567173,0.311746,9.791012e-01
436,VITAMIN D,CHOKING,4.0,8.530393,-0.915649,0.468912,34.0,842.0,0.087652,0.897484,0.950323,0.085266,0.251954,9.466459e-01
437,VITAMIN C,CHOKING,3.0,7.275924,-1.018042,0.412319,29.0,842.0,0.083580,0.881633,0.929263,0.126722,0.216191,9.497554e-01


In [19]:
results.signals # cut off at the decision threshold

,Product,Adverse Event,Count,Expected Count,log2,count/expected,product margin,event margin,fdr,FNR,Se,Sp,LowerBound,p_value
0,REUMOFAN PLUS,WEIGHT INCREASED,16.0,0.406436,4.539833,39.366569,44.0,31.0,0.079960,0.904489,0.879821,0.161140,15.686438,1.943874e-17
1,REUMOFAN PLUS,IMMOBILE,6.0,0.078665,4.191966,76.272727,44.0,6.0,0.080117,0.909704,0.861200,0.171486,10.161142,9.009766e-07
2,HYDROXYCUT REGULAR RAPID RELEASE CAPLETS,EMOTIONAL DISTRESS,19.0,0.896901,4.068400,21.184053,70.0,43.0,0.080021,0.900297,0.639618,0.422631,11.646773,2.541072e-17
3,"EMERGEN-C (ASCORBIC ACID, B-COMPLEX, ELECTROLY...",COUGH,6.0,0.144815,4.002402,41.432099,6.0,81.0,0.089050,0.925122,0.329246,0.568923,8.882635,2.815701e-06
4,HYDROXYCUT HARDCORE CAPSULES,MULTIPLE INJURIES,5.0,0.092372,3.966694,54.129032,31.0,10.0,0.086888,0.899664,0.572963,0.491910,8.271694,2.278501e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,"CENTRUM SILVER ULTRA WOMEN'S (MULTIMINERALS, M...",CHOKING,22.0,7.275924,1.391503,3.023671,29.0,842.0,0.115992,0.922704,0.124253,0.754106,1.824771,2.930835e-05
249,"EMERGEN-C (ASCORBIC ACID, B-COMPLEX, ELECTROLY...",CHOKING,6.0,1.505364,1.348375,3.985748,6.0,842.0,0.089672,0.924080,0.326736,0.578372,1.227613,1.849350e-02
250,CENTRUM SILVER ULTRA WOMENS MULTIMINERALS MULT...,CHOKING,6.0,1.505364,1.348375,3.985748,6.0,842.0,0.099235,0.921573,0.160521,0.735797,1.227613,1.849350e-02
251,CENTRUM SILVER WOMENS 50 PLUS MULTIMINERALS MU...,DYSPHAGIA,4.0,0.849225,1.348113,4.710175,10.0,285.0,0.086598,0.925929,0.261354,0.617815,1.024587,4.563272e-02


The signals and values are identical to those from openEBGM (but run quicker now!)

## Multi-item processor

I've also written code that implements a multi-item processor, to calculate drug-drug combinations and input these in the table as well  
However, this is currently *experimental* and needs some detailed testing.  
  
The steps of the algorithm are:
- we start off with table where each column has an id (id_label), an adverse event (ae_label) and a product (prod_label). The nature of SRS means that you have multiple rows with the same 
    id/AE, but different drugs. We want a way to process this to be able to produce a table that takes into account 
    potential pairs of drugs (and higher order effects). 
- We create a series of columns, all of which start with the string Drug to show all of the different drugs that occur on a single AE/id combo. 
- We then calculate all the possible combinations of the variables in these columns, creating it as a new column. 
- Optionally, we limit the size of the combinations at this point to 2 (because higher order interactions mean drastically many more combinations in the table) 
- We then explode out this column, to produce the necessary drugs and drug pairs that correspond to each id and adverse event as a new column 
    
All these steps are implemented in the `convert_multi_item_pipeline` function, which carries out this algorithm. Once you have the drug-drug pairs, you can then just use them in the MGPS algorithm in the exact same way as if you didn't pair them up. So after that, you put it in the `convert` function and then use the GPS algorithm the exact same way

In [20]:
help(convert_multi_item_pipeline)

Help on function convert_multi_item_pipeline in module vigipy.utils.data_prep:

convert_multi_item_pipeline(input_data, id_label='id', prod_label='name', ae_label='ae', count_label='count', limit_tuple_size=True, tuple_size_limit=2, count_unique_ids=True, opt_count=True)
    Processes and transforms input data to generate counts of drug combinations based on specified parameters.
    
    This function performs the following steps:
    1. Removes duplicate rows based on the specified columns (`id_label`, `prod_label`, `ae_label`).
    2. Processes the deduplicated data to generate and filter drug combinations, converting them into a format suitable for counting.
    3. Renames the column containing combinations to match the `prod_label`.
    4. Uses the `convert` function to count the occurrences of each combination, considering options for counting unique IDs and optimization.
    
    Parameters:
    input_data (pd.DataFrame): The input DataFrame containing data with columns for iden

In [21]:
help(multi_item_processing)

Help on function multi_item_processing in module vigipy.utils.data_prep:

multi_item_processing(input_data, id_label='id', prod_label='name', ae_label='AE', count_label='count', limit_tuple_sizes=True, tuple_size_limit=2)
    Processes a DataFrame to generate and format all possible permutations of drug names grouped by 
    specified identifiers and conditions. 
    
    The algorithm here is:
    1) we start off with table where each column has an id (id_label), an adverse event (ae_label) and a product (prod_label). The nature of SRS means that you have multiple rows with the same 
    id/AE, but different drugs. We want a way to process this to be able to produce a table that takes into account 
    potential pairs of drugs (and higher order effects). 
    2) We create a series of columns, all of which start with the string Drug to show all of the different drugs that occur on a single AE/id combo. 
    3) We then calculate all the possible combinations of the variables in these co

#### Demonstration

In [22]:
# Here is a demonstration of that function on a really small dataset

df = pd.DataFrame(
        {
            "id": [1, 1, 1, 1, 1, 2, 3, 4, 4, 4],
            "name": ["d", "a", "c", "b", "e", "a", "b", "c", "a", "b"],
            "AE": ["alpha", "alpha", "alpha", "alpha", "beta", "beta", "gamma", "delta", "alpha", "alpha"],
        }
    )

df['count'] = 1

df # see that patient '1' is taking 5 drugs (a,b,c,d,e) and experiencing 1 AE (alpha)

,id,name,AE,count
0,1,d,alpha,1
1,1,a,alpha,1
2,1,c,alpha,1
3,1,b,alpha,1
4,1,e,beta,1
5,2,a,beta,1
6,3,b,gamma,1
7,4,c,delta,1
8,4,a,alpha,1
9,4,b,alpha,1


In [23]:
# Takes reports that come in as drug:ae lists for each id, and splits them into a new data structure

id_label = 'id'
ae_label = 'AE'
prod_label = 'name'
clean_df = convert_multi_item_pipeline(df, id_label, prod_label, ae_label)


In [24]:
clean_df.contingency # and you now have the combination events as well!

AE,alpha,beta,delta,gamma
name,,,,
a,2,1,0,0
a | b,2,0,0,0
a | c,1,0,0,0
a | d,1,0,0,0
b,2,0,0,1
b | c,1,0,0,0
b | d,1,0,0,0
c,1,0,1,0
c | d,1,0,0,0


In [25]:
clean_df.data.sort_values(by='AE') # processed data showing that the combination data now also has an effect

,events,product_aes,count_across_brands,AE,name
0,2,3,2,alpha,a
2,2,2,2,alpha,a | b
3,1,1,2,alpha,a | c
4,1,1,2,alpha,a | d
5,2,3,2,alpha,b
7,1,1,2,alpha,b | c
8,1,1,2,alpha,b | d
9,1,2,2,alpha,c
11,1,1,2,alpha,c | d
12,1,1,2,alpha,d


#### Using this multi-item processor on CAERS

In [26]:
caers_dataset = "C:/Users/damlteam/Documents/vigipy_devops/vigipy/example_notebooks/test_datasets/caers_dataset.csv"

df = pd.read_csv(caers_dataset, header = 0)
df.rename(columns={'var1': 'name'}, inplace=True)
df.rename(columns={'var2': 'AE'}, inplace=True)

# drop duplicates
df = df.drop_duplicates(subset=['id', 'name', 'AE'], keep='first')

id_label='id'
ae_label='AE'
prod_label='name' 

clean_df = convert_multi_item_pipeline(df, id_label, prod_label, ae_label, tuple_size_limit=2) 

In [27]:
clean_df.data 
# original CAERS was 17189 elements, this is 39651 as we now have combinations appearing (that you can see too!)

,events,product_aes,count_across_brands,AE,name
0,1,1,121,HEART RATE INCREASED,1-PHENYLALANINE
1,1,1,121,HEART RATE INCREASED,1-PHENYLALANINE | EPA DHA
2,1,1,121,HEART RATE INCREASED,1-PHENYLALANINE | MAGNESIUM
3,1,1,121,HEART RATE INCREASED,1-PHENYLALANINE | MULTIMINERAL
4,1,1,121,HEART RATE INCREASED,1-PHENYLALANINE | NATTOZIMES
...,...,...,...,...,...
39646,1,1,70,PALPITATIONS,ZINC LOZENGE
39647,1,1,35,CEREBROVASCULAR ACCIDENT,ZIPFIZZ
39648,1,1,49,HYPOAESTHESIA,ZIPFIZZ
39649,1,1,35,CHEST DISCOMFORT,ZXT GOLD BEE POLLEN CAPSULES


In [28]:
## using the same GPS call as we used on earlier CAERS dataset, to see how it differs
time1 = time.time()
results_multi = gps(
    clean_df, # container that was processed with convert
    relative_risk=1, # relative risk parameter
    min_events=3, # minimum number of events for something to be considered a signal 
    decision_metric='rank', # what should we rank signals by? 'rank' means by the ranking_statistic later 
    decision_thres=log2_ebgm, # minimum value of the ranking statistic for something to be considered a signal
    ranking_statistic='log2', # which ranking statistic to use, here log2 means the log2 of the EBGM score
    truncate=True, # whether to truncate or not
    truncate_thres=1,  # threshold for truncation
    prior_init={"alpha1": 0.2041, "beta1": 0.05816, "alpha2": 1.415, "beta2": 1.838, "w": 0.0969}, # initial guesses for priors
    prior_param=None, # feed in an array if you want to just use those priors and do no optimisation
    expected_method="mantel-haentzel", # method for calculating expected counts
    method_alpha=1, # parameter for other methods of calculating expected counts
    minimization_method="Nelder-Mead", # which minimisation algorithm from scipy to use!
    minimization_bounds=((EPS, 20), (EPS, 10), (EPS, 20), (EPS, 10), (EPS, 1)), # bounds on minimisation: will only be applied to certain algorithms
    minimization_options=None, # any supplementary options for the minimiser
    message=True, # whether to be verbose and print messages
    opt_likelihood=True, # whether to use the optimised versions of likelihood functions (really should always be true)
    number_of_iterations=1000, # number of iterations of the optimiser
    tol_value=1.0e-6, # tolerance for the optimiser
    sim_anneal=False, # whether to use simulated annealing optimisation: very experimental
    product_label='name', # the name of the column in the processed data that contains the product names
    ae_label='AE' # the name of the column in the processed data that contains the AE names
)
time2 = time.time()
print("TIME TAKEN TO PRODUCE DATA = ", time2-time1)

BEGINNING HYPERPARAMETER OPTIMISATION
OPTIMISED PRIORS REACHED:  [3.10223352 0.66627994 5.40695926 6.6880516  0.12500349]
OPTIMISED FUNCTION VALUE =  5467.573019906686
Optimization terminated successfully.
CALCULATING EBGM SCORES
CALCULATING QUANTILES
GENERATING REPORT
TIME TAKEN TO PRODUCE DATA =  24.416024684906006


In [29]:
results_multi.signals # multi processor signals

,Product,Adverse Event,Count,Expected Count,log2,count/expected,product margin,event margin,fdr,FNR,Se,Sp,LowerBound,p_value
0,REUMOFAN PLUS,WEIGHT INCREASED,16.0,0.406436,4.116309,39.366569,44.0,31.0,0.129571,0.811414,0.894168,0.173705,11.675085,8.785534e-16
1,HYDROXYCUT REGULAR RAPID RELEASE CAPLETS,EMOTIONAL DISTRESS,19.0,0.896901,3.788752,21.184053,70.0,43.0,0.131788,0.824658,0.653464,0.438073,9.581493,2.887301e-16
2,REUMOFAN PLUS,IMMOBILE,6.0,0.078665,3.529881,76.272727,44.0,6.0,0.129245,0.822087,0.876790,0.185408,6.397147,9.917830e-06
3,HYDROXYCUT REGULAR RAPID RELEASE CAPLETS,INJURY,11.0,0.563170,3.468077,19.532275,70.0,27.0,0.131293,0.834189,0.675926,0.390925,6.948484,5.780109e-09
4,HYDROXYCUT HARDCORE CAPSULES,CARDIO-RESPIRATORY DISTRESS,8.0,0.304827,3.449038,26.244379,31.0,33.0,0.154170,0.838516,0.518327,0.533832,6.429929,7.017109e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,CENTRUM SILVER WOMEN'S 50 PLUS MULTIMINERALS M...,THROAT IRRITATION,3.0,0.581943,1.381002,5.155146,21.0,93.0,0.172427,0.869721,0.191493,0.717834,0.727306,1.487015e-01
285,CENTRUM SILVER ULTRA WOMENS MULTIMINERAL MULTI...,FOREIGN BODY TRAUMA,4.0,0.947557,1.370618,4.221384,12.0,265.0,0.182076,0.871776,0.147664,0.744054,0.807401,1.127710e-01
286,HERBALIFE CELL ACTIVATOR,VOMITING,4.0,0.951132,1.363591,4.205514,14.0,228.0,0.171040,0.855612,0.438431,0.548187,0.805538,1.137550e-01
287,HERBALIFE MULTIVITAMIN COMPLEX FORMULA 2,VOMITING,4.0,0.951132,1.363591,4.205514,14.0,228.0,0.171445,0.852891,0.448751,0.547544,0.805538,1.137550e-01


In [30]:
results.signals

,Product,Adverse Event,Count,Expected Count,log2,count/expected,product margin,event margin,fdr,FNR,Se,Sp,LowerBound,p_value
0,REUMOFAN PLUS,WEIGHT INCREASED,16.0,0.406436,4.539833,39.366569,44.0,31.0,0.079960,0.904489,0.879821,0.161140,15.686438,1.943874e-17
1,REUMOFAN PLUS,IMMOBILE,6.0,0.078665,4.191966,76.272727,44.0,6.0,0.080117,0.909704,0.861200,0.171486,10.161142,9.009766e-07
2,HYDROXYCUT REGULAR RAPID RELEASE CAPLETS,EMOTIONAL DISTRESS,19.0,0.896901,4.068400,21.184053,70.0,43.0,0.080021,0.900297,0.639618,0.422631,11.646773,2.541072e-17
3,"EMERGEN-C (ASCORBIC ACID, B-COMPLEX, ELECTROLY...",COUGH,6.0,0.144815,4.002402,41.432099,6.0,81.0,0.089050,0.925122,0.329246,0.568923,8.882635,2.815701e-06
4,HYDROXYCUT HARDCORE CAPSULES,MULTIPLE INJURIES,5.0,0.092372,3.966694,54.129032,31.0,10.0,0.086888,0.899664,0.572963,0.491910,8.271694,2.278501e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,"CENTRUM SILVER ULTRA WOMEN'S (MULTIMINERALS, M...",CHOKING,22.0,7.275924,1.391503,3.023671,29.0,842.0,0.115992,0.922704,0.124253,0.754106,1.824771,2.930835e-05
249,"EMERGEN-C (ASCORBIC ACID, B-COMPLEX, ELECTROLY...",CHOKING,6.0,1.505364,1.348375,3.985748,6.0,842.0,0.089672,0.924080,0.326736,0.578372,1.227613,1.849350e-02
250,CENTRUM SILVER ULTRA WOMENS MULTIMINERALS MULT...,CHOKING,6.0,1.505364,1.348375,3.985748,6.0,842.0,0.099235,0.921573,0.160521,0.735797,1.227613,1.849350e-02
251,CENTRUM SILVER WOMENS 50 PLUS MULTIMINERALS MU...,DYSPHAGIA,4.0,0.849225,1.348113,4.710175,10.0,285.0,0.086598,0.925929,0.261354,0.617815,1.024587,4.563272e-02


### Comments
- Note that the hyperparameters and likelihood are different: you have different background data with all the combinations and so you do produce different parameters 
- But the signals are largely pretty similar
- The differences are essentially due to masking/amplification. When you have combinations of drugs, you flood the database with more reports with very low counts. This tends to reduce all of the signal values because now each of the AEs appear in many more times with the combinations. This is the signal masking that people talk about. You can see this in the datasets: though the signals are broadly quite similar, the `log2` values in the `results_multi` are smaller: they are being masked due to including the combinations
- A better idea might be to only include combinations if they appear in the database more than a threshold number of times to avoid this masking effect, or potentially compute them separatelty